In [1]:
from hyperparams import *
from rawdata_preprocessing import *
#from RandomForestClassifier import RF_Classifying
#from SVMClassifier import SVM_Classifying

import Bio
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from sklearn.svm import SVC
from sklearn.model_selection import validation_curve
from sklearn.model_selection import train_test_split, GridSearchCV, KFold

from xgboost import plot_importance
from xgboost import XGBClassifier

from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import accuracy_score

#from warnings import filterwarnings
#filterwarnings('ignore')

from Logger import *
'''
import logging
import datetime
import string
temp = string.Template("WOOJUNG LOG ${var}.log")
filename = temp.substitute(var=datetime.datetime.now())
logging.basicConfig(filename="WOOJUNG's LOG.log", level=logging.DEBUG)
logger = logging.getLogger(__name__)
formatter = logging.Formatter('[%(asctime)s][%(levelname)s][%(filename)s:%(lineno)s]')

streamHandler = logging.StreamHandler()
fileHandler = logging.FileHandler('./WOOJUNG\'s LOG.log')
fileHandler.setFormatter(formatter)
logger.setLevel(level=logging.DEBUG)
logger.addHandler(streamHandler)
logger.addHandler(fileHandler)
'''
def Voting_Classifying(X, y, KFOLD_TIME) :
    #GBC_best = GBC_Classifying(X, y, KFOLD_TIME)
    #RFC_best = RF_Classifying(X, y, KFOLD_TIME)
    #SVMC_best = SVM_Classifying(X, y, KFOLD_TIME)
    #XGBC_best = XGB_Classifying(X, y, KFOLD_TIME)
    RFC_best = RandomForestClassifier(random_state=2)
    SVMC_best = SVC(probability=True, random_state=2)
    GBC_best = GradientBoostingClassifier(random_state=2)
    XGBC_best = XGBClassifier(probability=True, random_state=2)
    
    VC = VotingClassifier(estimators=[('rfc', RFC_best), 
                                      ('svc', SVMC_best), 
                                      ('gbc', GBC_best), 
                                      ('xgb', XGBC_best)], 
                          voting='soft', n_jobs=-1, verbose=2)
    param_range = [0.1, 1.0]
    param_grid = {
        'gbc__n_estimators' : [100], 'gbc__max_depth' : [6], 'gbc__min_samples_leaf': [3], 'gbc__min_samples_split' : [2], 'gbc__learning_rate' : [0.05],
        'svc__kernel' : ['rbf'], 'svc__C' : param_range, 'svc__gamma': param_range, 'svc__random_state' : [2],
        'xgb__kernel' : ['rbf'], 'xgb__C' : param_range, 'xgb__gamma':[0.1], 'xgb__random_state' : [2], 'xgb__learning_rate' : [0.01], 'xgb__n_estimators' : [100], 'xgb__max_depth' : [4],
        'rfc__n_estimators' : [10],'rfc__max_depth' : [6],'rfc__min_samples_leaf' : [8],'rfc__min_samples_split' : [8], 'rfc__max_leaf_nodes' : [10]
    }
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 5)
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    gsVC = GridSearchCV(estimator=VC, param_grid=param_grid, cv=KFOLD_TIME)

    gsVC = gsVC.fit(X_train,y_train)
    score = gsVC.score(X_test, y_test)
    
    print('GridSearchCV를 이용한 최적 매개변수 점수 ==> {:.3f}'.format(gsVC.best_score_))
    print('GridSearchCV를 이용한 최적 매개변수 ==> {}'.format(gsVC.best_params_))
    print('GridSearchCV를 이용한 test점수 ==> {:.3f}'.format(score))
    print('GridSearchCV를 이용한 최고 성능 모델 ==> \n{}'.format(gsVC.best_estimator_))
    
    return score
"""
def GBC_Classifying(X, y, KFOLD_TIME) :
    print("=======================================================")
    print("GBC_Classifying ... ")
    gbrt = GradientBoostingClassifier(random_state = 0)
    '''
    param_grid = {
        'n_estimators' : [100, 200], 
        'max_depth' : [6,8,10,12], 
        'min_samples_leaf': [3,5,7,10], 
        'min_samples_split' : [2,3,5,10], 
        'learning_rate' : [0.05, 0.1, 0.2]
    }
    '''
    param_grid = {'n_estimators' : [100], 'max_depth' : [6], 'min_samples_leaf': [3], 'min_samples_split' : [2], 'learning_rate' : [0.05]}

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 11)
    gsGBRT = GridSearchCV(gbrt, param_grid = param_grid, cv=KFOLD_TIME,
                         scoring="accuracy", n_jobs=6, verbose=2)
    
    '''
    gsGBRT.fit(X_train, y_train)
    score = gsGBRT.score(X_test, y_test)
    print('GridSearchCV를 이용한 최적 매개변수 점수 ==> {:.3f}'.format(gsGBRT.best_score_))
    print('GridSearchCV를 이용한 최적 매개변수 ==> {}'.format(gsGBRT.best_params_))
    print('GridSearchCV를 이용한 test점수 ==> {:.3f}'.format(score))
    print('GridSearchCV를 이용한 최고 성능 모델 ==> \n{}'.format(gsGBRT.best_estimator_))
    '''
    return gsGBRT

def SVM_Classifying(X, y, KFOLD_TIME) :
    print("=======================================================")
    print("SVM_Classifying ... ")
    SVMC = SVC(probability=True)
    param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
    #param_range = [0.1, 1.0]
    '''
    param_grid = [
        {'kernel' : ['rbf'], 'C' : param_range, 'gamma': param_range, 'random_state' : [2] },
        {'kernel' = ['poly'], 'C' : param_range, 'gamma': param_range, 'random_state' : [2] },
        {'kernel' = ['linear'], 'C' : param_range, 'random_state' : [2] },
        {'kernel' = ['sigmoid'], 'C' : param_range, 'gamma': param_range, 'random_state' : [2] }
    ]
    '''
    param_grid = [
        {'kernel' : ['rbf'], 'C' : param_range, 'gamma': param_range, 'random_state' : [2] }  
    ]

    #['C', 'break_ties', 'cache_size', 'class_weight', 'coef0', 'decision_function_shape', 'degree', 'gamma', 'kernel', 'max_iter', 'probability', 'random_state', 'shrinking', 'tol', 'verbose']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 11)
    gsSVMC = GridSearchCV(SVMC, param_grid = param_grid, cv=KFOLD_TIME,
                         scoring="accuracy", n_jobs=6, verbose=2)


    '''
    gsSVMC.fit(X, y)
    score = gsSVMC.score(X_test, y_test)

    SVMC_best = gsSVMC.best_estimator_

    print("best score : {}".format(gsSVMC.best_score_))
    print("best parameters : {}".format(gsSVMC.best_params_))
    print("train set score : {}".format(gsSVMC.score(X, y)))
    '''
    return gsSVMC

def XGB_Classifying(X, y, KFOLD_TIME) :
    print("=======================================================")
    print("XGB_Classifying ...")
    XGBC = XGBClassifier(probability=True)
    param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
    '''
    param_grid = {'kernel' : ['rbf'], 'C' : param_range, 'gamma': param_range, 'random_state' : [2],
                  'learning_rate' : [0.01, 0.05, 0.1, 0.15, 0.2], 
                  'n_estimators' : [100, 200, 400, 600], 
                  'max_depth' : [4,6,8,10,12] }
    '''
    param_grid = {'kernel' : ['rbf'], 'C' : param_range, 'gamma':[0.1], 'random_state' : [2],
                  'learning_rate' : [0.01], 
                  'n_estimators' : [100], 
                  'max_depth' : [4]}
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 11)
    X_train = np.array(X_train)
    X_test = np.array(X_test)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    
    gsXGBC = GridSearchCV(XGBC, param_grid = param_grid, cv=KFOLD_TIME,
                         scoring="accuracy", n_jobs=6, verbose=2)
    '''
    gsXGBC.fit(X_train, y_train)
    score = gsXGBC.score(X_test, y_test)
    print('GridSearchCV를 이용한 최적 매개변수 점수 ==> {:.3f}'.format(gsXGBC.best_score_))
    print('GridSearchCV를 이용한 최적 매개변수 ==> {}'.format(gsXGBC.best_params_))
    print('GridSearchCV를 이용한 test점수 ==> {:.3f}'.format(score))
    print('GridSearchCV를 이용한 최고 성능 모델 ==> \n{}'.format(gsXGBC.best_estimator_))
    '''
    return gsXGBC
"""
def RF_Classifying(X, y, KFOLD_TIME, param_grid) :
    logger.debug("=======================================================")
    logger.debug("RF_Classifying ... ")
    RFC = RandomForestClassifier(random_state=2)
    '''
    param_grid = { 'n_estimators' : [10, 100],
           'max_depth' : [6, 8, 10, 12],
           'min_samples_leaf' : [8, 12, 18],
           'min_samples_split' : [8, 16, 20], 
            'max_leaf_nodes' : [1,10] 
    }
    '''
    #param_grid = {'n_estimators' : [10],'max_depth' : [6],'min_samples_leaf' : [8],'min_samples_split' : [8], 'max_leaf_nodes' : [10]}

    kfold = KFold(n_splits=KFOLD_TIME, shuffle=True, random_state=11)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 11)
    
    gsRFC = GridSearchCV(RFC, param_grid = param_grid, cv=kfold,
                         scoring="accuracy", n_jobs=-1, verbose=2)
    
    gsRFC.fit(X_train, y_train)
    # X_train, y_train 을 0.8:0.2 이런식으로 나눠서 cv 시킨다는 뜻인듯 ?
    
    score = gsRFC.score(X_test, y_test)
    logger.debug('GridSearchCV를 이용한 최적 매개변수 점수 ==> {:.3f}'.format(gsRFC.best_score_))
    logger.debug('GridSearchCV를 이용한 최적 매개변수 ==> {}'.format(gsRFC.best_params_))
    logger.debug('GridSearchCV를 이용한 test점수 ==> {:.3f}'.format(score))
    logger.debug('GridSearchCV를 이용한 최고 성능 모델 ==> \n{}'.format(gsRFC.best_estimator_))
    
    #return gsRFC
    return score

def LR_Classifying(X, y, KFOLD_TIME, param_grid) :
    logger.debug("=======================================================")
    logger.debug("LogisticRegressionCV_Classifying ... ")
    
    LRC = LogisticRegressionCV(random_state = 2)
    param_range = [0.01 ,0.1, 1, 10, 100]
    param_grid = {'C' : param_range, 'gamma':[0.1], 'random_state' : [2],
                  'learning_rate' : [0.01], 
                  'n_estimators' : [100], 
                  'max_depth' : [4]}
    gsLRC = GridSearchCV(LRC, param_grid = param_grid, cv=KFOLD_TIME,
                         scoring="accuracy", n_jobs=-1, verbose=2)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 11)
    gsLRC.fit(X_train, y_train)
    
    score = gsLRC.score(X_test, y_test)
    
    
    score = gsLRC.score(X_test, y_test)
    logger.debug('GridSearchCV를 이용한 최적 매개변수 점수 ==> {:.3f}'.format(gsLRC.best_score_))
    logger.debug('GridSearchCV를 이용한 최적 매개변수 ==> {}'.format(gsLRC.best_params_))
    logger.debug('GridSearchCV를 이용한 test점수 ==> {:.3f}'.format(score))
    logger.debug('GridSearchCV를 이용한 최고 성능 모델 ==> \n{}'.format(gsLRC.best_estimator_))
    return score

KFOLD_TIME = 5

p_feature_names = ['A', 'AA', 'AAA', 'AAB', 'AAC', 'AAD', 'AAE', 'AAF', 'AAG', 'AB', 'ABA', 'ABB', 'ABC', 'ABD', 'ABE', 'ABF', 'ABG', 'AC', 'ACA', 'ACB', 'ACC', 'ACD', 'ACE', 'ACF', 'ACG', 'AD', 'ADA', 'ADB', 'ADC', 'ADD', 'ADE', 'ADF', 'ADG', 'AE', 'AEA', 'AEB', 'AEC', 'AED', 'AEE', 'AEF', 'AEG', 'AF', 'AFA', 'AFB', 'AFC', 'AFD', 'AFE', 'AFF', 'AFG', 'AG', 'AGA', 'AGB', 'AGC', 'AGD', 'AGE', 'AGF', 'AGG', 'B', 'BA', 'BAA', 'BAB', 'BAC', 'BAD', 'BAE', 'BAF', 'BAG', 'BB', 'BBA', 'BBB', 'BBC', 'BBD', 'BBE', 'BBF', 'BBG', 'BC', 'BCA', 'BCB', 'BCC', 'BCD', 'BCE', 'BCF', 'BCG', 'BD', 'BDA', 'BDB', 'BDC', 'BDD', 'BDE', 'BDF', 'BDG', 'BE', 'BEA', 'BEB', 'BEC', 'BED', 'BEE', 'BEF', 'BEG', 'BF', 'BFA', 'BFB', 'BFC', 'BFD', 'BFE', 'BFF', 'BFG', 'BG', 'BGA', 'BGB', 'BGC', 'BGD', 'BGE', 'BGF', 'BGG', 'C', 'CA', 'CAA', 'CAB', 'CAC', 'CAD', 'CAE', 'CAF', 'CAG', 'CB', 'CBA', 'CBB', 'CBC', 'CBD', 'CBE', 'CBF', 'CBG', 'CC', 'CCA', 'CCB', 'CCC', 'CCD', 'CCE', 'CCF', 'CCG', 'CD', 'CDA', 'CDB', 'CDC', 'CDD', 'CDE', 'CDF', 'CDG', 'CE', 'CEA', 'CEB', 'CEC', 'CED', 'CEE', 'CEF', 'CEG', 'CF', 'CFA', 'CFB', 'CFC', 'CFD', 'CFE', 'CFF', 'CFG', 'CG', 'CGA', 'CGB', 'CGC', 'CGD', 'CGE', 'CGF', 'CGG', 'D', 'DA', 'DAA', 'DAB', 'DAC', 'DAD', 'DAE', 'DAF', 'DAG', 'DB', 'DBA', 'DBB', 'DBC', 'DBD', 'DBE', 'DBF', 'DBG', 'DC', 'DCA', 'DCB', 'DCC', 'DCD', 'DCE', 'DCF', 'DCG', 'DD', 'DDA', 'DDB', 'DDC', 'DDD', 'DDE', 'DDF', 'DDG', 'DE', 'DEA', 'DEB', 'DEC', 'DED', 'DEE', 'DEF', 'DEG', 'DF', 'DFA', 'DFB', 'DFC', 'DFD', 'DFE', 'DFF', 'DFG', 'DG', 'DGA', 'DGB', 'DGC', 'DGD', 'DGE', 'DGF', 'DGG', 'E', 'EA', 'EAA', 'EAB', 'EAC', 'EAD', 'EAE', 'EAF', 'EAG', 'EB', 'EBA', 'EBB', 'EBC', 'EBD', 'EBE', 'EBF', 'EBG', 'EC', 'ECA', 'ECB', 'ECC', 'ECD', 'ECE', 'ECF', 'ECG', 'ED', 'EDA', 'EDB', 'EDC', 'EDD', 'EDE', 'EDF', 'EDG', 'EE', 'EEA', 'EEB', 'EEC', 'EED', 'EEE', 'EEF', 'EEG', 'EF', 'EFA', 'EFB', 'EFC', 'EFD', 'EFE', 'EFF', 'EFG', 'EG', 'EGA', 'EGB', 'EGC', 'EGD', 'EGE', 'EGF', 'EGG', 'F', 'FA', 'FAA', 'FAB', 'FAC', 'FAD', 'FAE', 'FAF', 'FAG', 'FB', 'FBA', 'FBB', 'FBC', 'FBD', 'FBE', 'FBF', 'FBG', 'FC', 'FCA', 'FCB', 'FCC', 'FCD', 'FCE', 'FCF', 'FCG', 'FD', 'FDA', 'FDB', 'FDC', 'FDD', 'FDE', 'FDF', 'FDG', 'FE', 'FEA', 'FEB', 'FEC', 'FED', 'FEE', 'FEF', 'FEG', 'FF', 'FFA', 'FFB', 'FFC', 'FFD', 'FFE', 'FFF', 'FFG', 'FG', 'FGA', 'FGB', 'FGC', 'FGD', 'FGE', 'FGF', 'FGG', 'G', 'GA', 'GAA', 'GAB', 'GAC', 'GAD', 'GAE', 'GAF', 'GAG', 'GB', 'GBA', 'GBB', 'GBC', 'GBD', 'GBE', 'GBF', 'GBG', 'GC', 'GCA', 'GCB', 'GCC', 'GCD', 'GCE', 'GCF', 'GCG', 'GD', 'GDA', 'GDB', 'GDC', 'GDD', 'GDE', 'GDF', 'GDG', 'GE', 'GEA', 'GEB', 'GEC', 'GED', 'GEE', 'GEF', 'GEG', 'GF', 'GFA', 'GFB', 'GFC', 'GFD', 'GFE', 'GFF', 'GFG', 'GG', 'GGA', 'GGB', 'GGC', 'GGD', 'GGE', 'GGF', 'GGG']
r_feature_names = ['A', 'AA', 'AAA', 'AAAA', 'AAAC', 'AAAG', 'AAAU', 'AAC', 'AACA', 'AACC', 'AACG', 'AACU', 'AAG', 'AAGA', 'AAGC', 'AAGG', 'AAGU', 'AAU', 'AAUA', 'AAUC', 'AAUG', 'AAUU', 'AC', 'ACA', 'ACAA', 'ACAC', 'ACAG', 'ACAU', 'ACC', 'ACCA', 'ACCC', 'ACCG', 'ACCU', 'ACG', 'ACGA', 'ACGC', 'ACGG', 'ACGU', 'ACU', 'ACUA', 'ACUC', 'ACUG', 'ACUU', 'AG', 'AGA', 'AGAA', 'AGAC', 'AGAG', 'AGAU', 'AGC', 'AGCA', 'AGCC', 'AGCG', 'AGCU', 'AGG', 'AGGA', 'AGGC', 'AGGG', 'AGGU', 'AGU', 'AGUA', 'AGUC', 'AGUG', 'AGUU', 'AU', 'AUA', 'AUAA', 'AUAC', 'AUAG', 'AUAU', 'AUC', 'AUCA', 'AUCC', 'AUCG', 'AUCU', 'AUG', 'AUGA', 'AUGC', 'AUGG', 'AUGU', 'AUU', 'AUUA', 'AUUC', 'AUUG', 'AUUU', 'C', 'CA', 'CAA', 'CAAA', 'CAAC', 'CAAG', 'CAAU', 'CAC', 'CACA', 'CACC', 'CACG', 'CACU', 'CAG', 'CAGA', 'CAGC', 'CAGG', 'CAGU', 'CAU', 'CAUA', 'CAUC', 'CAUG', 'CAUU', 'CC', 'CCA', 'CCAA', 'CCAC', 'CCAG', 'CCAU', 'CCC', 'CCCA', 'CCCC', 'CCCG', 'CCCU', 'CCG', 'CCGA', 'CCGC', 'CCGG', 'CCGU', 'CCU', 'CCUA', 'CCUC', 'CCUG', 'CCUU', 'CG', 'CGA', 'CGAA', 'CGAC', 'CGAG', 'CGAU', 'CGC', 'CGCA', 'CGCC', 'CGCG', 'CGCU', 'CGG', 'CGGA', 'CGGC', 'CGGG', 'CGGU', 'CGU', 'CGUA', 'CGUC', 'CGUG', 'CGUU', 'CU', 'CUA', 'CUAA', 'CUAC', 'CUAG', 'CUAU', 'CUC', 'CUCA', 'CUCC', 'CUCG', 'CUCU', 'CUG', 'CUGA', 'CUGC', 'CUGG', 'CUGU', 'CUU', 'CUUA', 'CUUC', 'CUUG', 'CUUU', 'G', 'GA', 'GAA', 'GAAA', 'GAAC', 'GAAG', 'GAAU', 'GAC', 'GACA', 'GACC', 'GACG', 'GACU', 'GAG', 'GAGA', 'GAGC', 'GAGG', 'GAGU', 'GAU', 'GAUA', 'GAUC', 'GAUG', 'GAUU', 'GC', 'GCA', 'GCAA', 'GCAC', 'GCAG', 'GCAU', 'GCC', 'GCCA', 'GCCC', 'GCCG', 'GCCU', 'GCG', 'GCGA', 'GCGC', 'GCGG', 'GCGU', 'GCU', 'GCUA', 'GCUC', 'GCUG', 'GCUU', 'GG', 'GGA', 'GGAA', 'GGAC', 'GGAG', 'GGAU', 'GGC', 'GGCA', 'GGCC', 'GGCG', 'GGCU', 'GGG', 'GGGA', 'GGGC', 'GGGG', 'GGGU', 'GGU', 'GGUA', 'GGUC', 'GGUG', 'GGUU', 'GU', 'GUA', 'GUAA', 'GUAC', 'GUAG', 'GUAU', 'GUC', 'GUCA', 'GUCC', 'GUCG', 'GUCU', 'GUG', 'GUGA', 'GUGC', 'GUGG', 'GUGU', 'GUU', 'GUUA', 'GUUC', 'GUUG', 'GUUU', 'U', 'UA', 'UAA', 'UAAA', 'UAAC', 'UAAG', 'UAAU', 'UAC', 'UACA', 'UACC', 'UACG', 'UACU', 'UAG', 'UAGA', 'UAGC', 'UAGG', 'UAGU', 'UAU', 'UAUA', 'UAUC', 'UAUG', 'UAUU', 'UC', 'UCA', 'UCAA', 'UCAC', 'UCAG', 'UCAU', 'UCC', 'UCCA', 'UCCC', 'UCCG', 'UCCU', 'UCG', 'UCGA', 'UCGC', 'UCGG', 'UCGU', 'UCU', 'UCUA', 'UCUC', 'UCUG', 'UCUU', 'UG', 'UGA', 'UGAA', 'UGAC', 'UGAG', 'UGAU', 'UGC', 'UGCA', 'UGCC', 'UGCG', 'UGCU', 'UGG', 'UGGA', 'UGGC', 'UGGG', 'UGGU', 'UGU', 'UGUA', 'UGUC', 'UGUG', 'UGUU', 'UU', 'UUA', 'UUAA', 'UUAC', 'UUAG', 'UUAU', 'UUC', 'UUCA', 'UUCC', 'UUCG', 'UUCU', 'UUG', 'UUGA', 'UUGC', 'UUGG', 'UUGU', 'UUU', 'UUUA', 'UUUC', 'UUUG', 'UUUU']
xgb_r_feature_names = ['r_A', 'r_AA', 'r_AAA', 'r_AAAA', 'r_AAAC', 'r_AAAG', 'r_AAAU', 'r_AAC', 'r_AACA', 'r_AACC', 'r_AACG', 'r_AACU', 'r_AAG', 'r_AAGA', 'r_AAGC', 'r_AAGG', 'r_AAGU', 'r_AAU', 'r_AAUA', 'r_AAUC', 'r_AAUG', 'r_AAUU', 'r_AC', 'r_ACA', 'r_ACAA', 'r_ACAC', 'r_ACAG', 'r_ACAU', 'r_ACC', 'r_ACCA', 'r_ACCC', 'r_ACCG', 'r_ACCU', 'r_ACG', 'r_ACGA', 'r_ACGC', 'r_ACGG', 'r_ACGU', 'r_ACU', 'r_ACUA', 'r_ACUC', 'r_ACUG', 'r_ACUU', 'r_AG', 'r_AGA', 'r_AGAA', 'r_AGAC', 'r_AGAG', 'r_AGAU', 'r_AGC', 'r_AGCA', 'r_AGCC', 'r_AGCG', 'r_AGCU', 'r_AGG', 'r_AGGA', 'r_AGGC', 'r_AGGG', 'r_AGGU', 'r_AGU', 'r_AGUA', 'r_AGUC', 'r_AGUG', 'r_AGUU', 'r_AU', 'r_AUA', 'r_AUAA', 'r_AUAC', 'r_AUAG', 'r_AUAU', 'r_AUC', 'r_AUCA', 'r_AUCC', 'r_AUCG', 'r_AUCU', 'r_AUG', 'r_AUGA', 'r_AUGC', 'r_AUGG', 'r_AUGU', 'r_AUU', 'r_AUUA', 'r_AUUC', 'r_AUUG', 'r_AUUU', 'r_C', 'r_CA', 'r_CAA', 'r_CAAA', 'r_CAAC', 'r_CAAG', 'r_CAAU', 'r_CAC', 'r_CACA', 'r_CACC', 'r_CACG', 'r_CACU', 'r_CAG', 'r_CAGA', 'r_CAGC', 'r_CAGG', 'r_CAGU', 'r_CAU', 'r_CAUA', 'r_CAUC', 'r_CAUG', 'r_CAUU', 'r_CC', 'r_CCA', 'r_CCAA', 'r_CCAC', 'r_CCAG', 'r_CCAU', 'r_CCC', 'r_CCCA', 'r_CCCC', 'r_CCCG', 'r_CCCU', 'r_CCG', 'r_CCGA', 'r_CCGC', 'r_CCGG', 'r_CCGU', 'r_CCU', 'r_CCUA', 'r_CCUC', 'r_CCUG', 'r_CCUU', 'r_CG', 'r_CGA', 'r_CGAA', 'r_CGAC', 'r_CGAG', 'r_CGAU', 'r_CGC', 'r_CGCA', 'r_CGCC', 'r_CGCG', 'r_CGCU', 'r_CGG', 'r_CGGA', 'r_CGGC', 'r_CGGG', 'r_CGGU', 'r_CGU', 'r_CGUA', 'r_CGUC', 'r_CGUG', 'r_CGUU', 'r_CU', 'r_CUA', 'r_CUAA', 'r_CUAC', 'r_CUAG', 'r_CUAU', 'r_CUC', 'r_CUCA', 'r_CUCC', 'r_CUCG', 'r_CUCU', 'r_CUG', 'r_CUGA', 'r_CUGC', 'r_CUGG', 'r_CUGU', 'r_CUU', 'r_CUUA', 'r_CUUC', 'r_CUUG', 'r_CUUU', 'r_G', 'r_GA', 'r_GAA', 'r_GAAA', 'r_GAAC', 'r_GAAG', 'r_GAAU', 'r_GAC', 'r_GACA', 'r_GACC', 'r_GACG', 'r_GACU', 'r_GAG', 'r_GAGA', 'r_GAGC', 'r_GAGG', 'r_GAGU', 'r_GAU', 'r_GAUA', 'r_GAUC', 'r_GAUG', 'r_GAUU', 'r_GC', 'r_GCA', 'r_GCAA', 'r_GCAC', 'r_GCAG', 'r_GCAU', 'r_GCC', 'r_GCCA', 'r_GCCC', 'r_GCCG', 'r_GCCU', 'r_GCG', 'r_GCGA', 'r_GCGC', 'r_GCGG', 'r_GCGU', 'r_GCU', 'r_GCUA', 'r_GCUC', 'r_GCUG', 'r_GCUU', 'r_GG', 'r_GGA', 'r_GGAA', 'r_GGAC', 'r_GGAG', 'r_GGAU', 'r_GGC', 'r_GGCA', 'r_GGCC', 'r_GGCG', 'r_GGCU', 'r_GGG', 'r_GGGA', 'r_GGGC', 'r_GGGG', 'r_GGGU', 'r_GGU', 'r_GGUA', 'r_GGUC', 'r_GGUG', 'r_GGUU', 'r_GU', 'r_GUA', 'r_GUAA', 'r_GUAC', 'r_GUAG', 'r_GUAU', 'r_GUC', 'r_GUCA', 'r_GUCC', 'r_GUCG', 'r_GUCU', 'r_GUG', 'r_GUGA', 'r_GUGC', 'r_GUGG', 'r_GUGU', 'r_GUU', 'r_GUUA', 'r_GUUC', 'r_GUUG', 'r_GUUU', 'r_U', 'r_UA', 'r_UAA', 'r_UAAA', 'r_UAAC', 'r_UAAG', 'r_UAAU', 'r_UAC', 'r_UACA', 'r_UACC', 'r_UACG', 'r_UACU', 'r_UAG', 'r_UAGA', 'r_UAGC', 'r_UAGG', 'r_UAGU', 'r_UAU', 'r_UAUA', 'r_UAUC', 'r_UAUG', 'r_UAUU', 'r_UC', 'r_UCA', 
                       'r_UCAA', 'UCAC', 'UCAG', 'UCAU', 'UCC', 'UCCA', 'UCCC', 'UCCG', 'UCCU', 'UCG', 'UCGA', 'UCGC', 'UCGG', 'UCGU', 'UCU', 'UCUA', 'UCUC', 'UCUG', 'UCUU', 'UG', 'UGA', 'UGAA', 'UGAC', 'UGAG', 'UGAU', 'UGC', 'UGCA', 'UGCC', 'UGCG', 'UGCU', 'UGG', 'UGGA', 'UGGC', 'UGGG', 'UGGU', 'UGU', 'UGUA', 'UGUC', 'UGUG', 'UGUU', 'UU', 'UUA', 'UUAA', 'UUAC', 'UUAG', 'UUAU', 'UUC', 'UUCA', 'UUCC', 'UUCG', 'UUCU', 'UUG', 'UUGA', 'UUGC', 'UUGG', 'UUGU', 'UUU', 'UUUA', 'UUUC', 'UUUG', 'UUUU']

cv=KFold(n_splits=3,random_state=5,shuffle=True)

train_combined_arr = []
val_combined_arr = []

def classify(npz_path, param_grid) :
    logger.debug("Dataset : {}".format(npz_path))
    mydata = np.load(npz_path)
    XP = mydata['XP']
    XR = mydata['XR']
    Y = mydata['Y']

    combined_pd = pd.DataFrame(data= np.c_[np.c_[XP, XR], Y], columns= p_feature_names + r_feature_names + ['target'])


    features = list(combined_pd.columns[:-1])
    X = combined_pd[features]
    y = combined_pd['target']

    #result = Voting_Classifying(X, y, KFOLD_TIME)
    result = RF_Classifying(X, y, KFOLD_TIME, param_grid)

    return result

def classify_and_print_NPInter():
    logger.debug("=======================================================")
    result = classify(NPZ_PATH["NPInter"], PARAM_GRID["NPInter"]["RFC"])
    logger.debug("K_fold with", KFOLD_TIME, "epoch : ", result)
    
def classify_and_print_RPI(size):
    logger.debug("=======================================================")
    result = classify(NPZ_PATH["RPI"][size], PARAM_GRID["RPI"][size]["RFC"])
    logger.debug(result)

if __name__ == "__main__":
    logger.debug("Classification is about to start ... ")
    classify_and_print_RPI(369)
    #classify_and_print_NPInter()
    classify_and_print_RPI(1807)
    classify_and_print_RPI(2241)
    #classify_and_print_RPI(369)
    classify_and_print_RPI(488)
    classify_and_print_NPInter()

Classification is about to start ... 
Dataset : npz/RPI369.npz
RF_Classifying ... 
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 1200 candidates, totalling 6000 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   10.5s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   19.1s
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:   28.8s
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:   40.2s
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed:   54.0s
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 4885 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 5816 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 6000 out of 6000 | elapsed:  2.8min finished
GridSearchCV를 이용한 최적 매개변수 점수 ==> 0.593
GridSearchCV를 이용한 최적 매개변수 ==> {'max_depth': 7, 'max_leaf_nodes': 20, 'min_samples_leaf': 10, 'min_samples_split': 22, 'n_estimators': 10}

Fitting 5 folds for each of 900 candidates, totalling 4500 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    4.7s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   28.1s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed: 12.3min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 15.3min
[Parallel(n_jobs=-1)]: Done 4500 out of 4500 | elapsed: 17.6min finished
GridSearchCV를 이용한 최적 매개변수 점수 ==> 0.952
GridSearchCV를 이용한 최적 매개변수 ==> {'max_depth': 7, 'max_leaf_nodes': 30, 'min_samples_leaf': 18, 'min_samples_split': 6, 'n_estimators': 100}
GridSearchCV를 이용한 test점수 ==> 0.934
GridSearchCV를 이용한 최고 성능 모델 ==> 
RandomForestClassifier(max_depth=7, max_leaf_nodes=30, min

...

[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:   33.4s
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed:   59.0s
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed:  3.0min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed:  4.0min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 3600 out of 3600 | elapsed:  5.5min finished
GridSearchCV를 이용한 최적 매개변수 점수 ==> 0.895
GridSearchCV를 이용한 최적 매개변수 ==> {'max_depth': 6, 'max_leaf_nodes': 10, 'min_samples_leaf': 4, 'min_samples_split': 13, 'n_estimators': 125}
GridSearchCV를 이용한 test점수 ==> 0.898
GridSearchCV를 이용한 최고 성능 모델 ==> 
RandomForestClassifier(max_depth=6, max_leaf_nodes=10, min_samples_leaf=4,
                       min_samples_split=13, n

RF_Classifying ... 
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Fitting 5 folds for each of 900 candidates, totalling 4500 fits


[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   44.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 349 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done 632 tasks      | elapsed: 19.9min
[Parallel(n_jobs=-1)]: Done 997 tasks      | elapsed: 35.5min
[Parallel(n_jobs=-1)]: Done 1442 tasks      | elapsed: 55.6min
[Parallel(n_jobs=-1)]: Done 1969 tasks      | elapsed: 82.0min
[Parallel(n_jobs=-1)]: Done 2576 tasks      | elapsed: 108.9min
[Parallel(n_jobs=-1)]: Done 3265 tasks      | elapsed: 148.2min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 204.0min
[Parallel(n_jobs=-1)]: Done 4500 out of 4500 | elapsed: 251.0min finished
GridSearchCV를 이용한 최적 매개변수 점수 ==> 0.898
GridSearchCV를 이용한 최적 매개변수 ==> {'max_depth': 7, 'max_leaf_nodes': 30, 'min_samples_leaf': 30, 'min_samples_split': 6, 'n_estimators': 50}
GridSearchCV를 이용한 test점수 ==> 0.906
GridSearchCV를 이용한 최고 성능 모델 ==> 
RandomForestClassifier(max_depth=7, max_leaf_nodes=30, 

K_fold with 5 epoch :  0.9058823529411765
